# Generate force coefficients

The following steps are performed to obtain all the artifacts for force coefficient use case:

1. Read parameters file
2. Read Mesh description
   * Its path can be specified
   * Or it can be obtained from the simulation configuration file
3. Read data for pressure coefficient

Instantiate path manager

In [1]:
from cfdmod.use_cases.pressure.path_manager import CfPathManager
import pathlib

path_manager = CfPathManager(output_path=pathlib.Path("./output/pressure"))

Read parameters file

In [2]:
from cfdmod.use_cases.pressure.force.Cf_config import CfCaseConfig

config_path = pathlib.Path("./fixtures/tests/pressure/Cf_params.yaml")
post_proc_cfg = CfCaseConfig.from_file(config_path)

post_proc_cfg.force_coefficient

{'measurement_1': CfConfig(statistics=[BasicStatisticModel(stats='mean'), BasicStatisticModel(stats='rms'), BasicStatisticModel(stats='skewness'), BasicStatisticModel(stats='kurtosis'), ParameterizedStatisticModel(stats='mean_eq', params=MeanEquivalentParamsModel(scale_factor=0.61)), ParameterizedStatisticModel(stats='min', params=ExtremeMovingAverageParamsModel(method_type='Moving Average', window_size_real_scale=3.0)), ParameterizedStatisticModel(stats='max', params=ExtremeAbsoluteParamsModel(method_type='Absolute'))], bodies=[BodyConfig(name='building2', sub_bodies=ZoningModel(x_intervals=[-inf, inf], y_intervals=[-inf, inf], z_intervals=[0.0, 10.0, 20.0]))], directions=['x', 'y', 'z'], transformation=TransformationConfig(translation=(0, 0, 0), rotation=(0.0, 0.0, 0.0), fixed_point=(0, 0, 0))),
 'measurement_2': CfConfig(statistics=[BasicStatisticModel(stats='mean'), BasicStatisticModel(stats='rms'), BasicStatisticModel(stats='skewness'), BasicStatisticModel(stats='kurtosis'), Param

Read Mesh from file

In [3]:
from lnas import LnasFormat

mesh_path = pathlib.Path("./fixtures/tests/pressure/galpao/galpao.normalized.lnas")
mesh = LnasFormat.from_file(mesh_path)

mesh.geometry.vertices.shape, mesh.geometry.triangles.shape

((1549, 3), (2915, 3))

Calculate force coefficient

In [4]:
from cfdmod.logger import logger
from cfdmod.use_cases.pressure.force.Cf_data import CommonOutput, process_Cf

cp_path = pathlib.Path("./fixtures/tests/pressure/data/cp_t.resampled.h5")

for cfg_label, cfg in post_proc_cfg.force_coefficient.items():
        logger.info(f"Processing Cf config {cfg_label} ...")

        cf_output_dict: dict[str, CommonOutput] = process_Cf(
            mesh=mesh,
            cfg=cfg,
            cp_path=cp_path,
            bodies_definition=post_proc_cfg.bodies,
            time_scale_factor=post_proc_cfg.time_scale_conversion.time_scale,
        )
        already_saved = False
        for direction_lbl, cf_output in cf_output_dict.items():
            path_manager.direction_label = direction_lbl
            cf_output.save_outputs(
                cfg_label=cfg_label, path_manager=path_manager, already_saved=already_saved
            )
            already_saved = True

        logger.info(f"Processed Cf config {cfg_label}!")


[2024-05-20 10:12:19,083] [INFO] - cfdmod - Processing Cf config measurement_1 ... (3221285436.py:7)
[2024-05-20 10:12:20,504] [INFO] - cfdmod - Processed Cf config measurement_1! (3221285436.py:24)
[2024-05-20 10:12:20,504] [INFO] - cfdmod - Processing Cf config measurement_2 ... (3221285436.py:7)
[2024-05-20 10:12:20,785] [INFO] - cfdmod - Processed Cf config measurement_2! (3221285436.py:24)
